In [5]:
# # Directory to Raw ground water levels folder
#The Grundwasserstand-Monatsmittel (monthly average groundwater level) directory is contained in the ehyd_messstellen_all_gw.zip file
directory = "/content/Grundwasserstand-Monatsmittel" # Adjust to corresponding folder on your device

# Folder Path for cleaned sampling points
cleaned_output_folder = 'sampling_points_cleaned'

# Folder path for the 487 sample points needed for prediction
folder_path_487 = "filtered/"

In [6]:
# Import libraries
import pandas as pd
import numpy as np
import os
import shutil
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [7]:
# Function to get SMAPE score
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

## **Data Cleaning Process**

In [8]:
# Renaming the raw files
import os

for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("Grundwasserstand-Monatsmittel-"):
        new_filename = filename.replace("Grundwasserstand-Monatsmittel-", "")
        old_filepath = os.path.join(directory, filename)
        new_filepath = os.path.join(directory, new_filename)
        os.rename(old_filepath, new_filepath)

In [ ]:
#ENCODING and cleaning the raw files to have just the metadata (months and their groundwater levels)

import os

def process_file(input_file, output_file, encoding='ISO-8859-1'):
    # Read the input file with a specific encoding
    try:
        with open(input_file, 'r', encoding=encoding) as f:
            lines = f.readlines()
    except UnicodeDecodeError:
        print(f"Error decoding {input_file} with encoding {encoding}.")
        return

    # Find the line starting with 'Werte:'
    start_index = None
    for i, line in enumerate(lines):
        if line.startswith('Werte:'):
            start_index = i
            break

    if start_index is None:
        print(f"Warning: 'Werte:' not found in {input_file}.")
        return

    relevant_lines = lines[start_index + 1:]

    with open(output_file, 'w', encoding=encoding) as f:
        f.writelines(relevant_lines)

    print(f"Processed {input_file} and saved to {output_file}.")

def process_folder(folder_path, output_folder, encoding='ISO-8859-1'):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            input_file = os.path.join(folder_path, filename)
            output_file = os.path.join(output_folder, filename)
            process_file(input_file, output_file, encoding)


process_folder(directory, cleaned_output_folder)

In [10]:
# Cleaning data format

for i in os.listdir(cleaned_output_folder):
  try:
    path=os.path.join(cleaned_output_folder, i)
    df=pd.read_csv(path, encoding='latin-1', header=None)
    df["data"]=df[0].str.split(';')
    df_=pd.DataFrame(df["data"].to_list(), columns=['Date', 'Data1', 'Data2'])
    df[1]=df[1].str.strip(' ;')
    df_[1]=df[1]
    df_["Levels"]=df_["Data1"]+"."+df_[1]
    df_=df_[["Date","Levels"]]
    df_["Levels"]=df_["Levels"].astype("float64")
    df_.to_csv(path, index=False)
  except:
    pass

In [11]:
# 487 points needed for predictions

points_data_487= ['330803.csv', '305714.csv', '335562.csv', '345330.csv', '301572.csv', '318584.csv', '307843.csv',
 '331116.csv', '329995.csv', '315671.csv', '308783.csv', '335984.csv', '306209.csv', '316026.csv',
 '334052.csv', '320754.csv', '326132.csv', '329573.csv', '322156.csv', '328724.csv', '345132.csv',
 '322396.csv', '325969.csv', '312447.csv', '332783.csv', '300269.csv', '300137.csv', '324095.csv',
 '304741.csv', '335588.csv', '327239.csv', '345694.csv', '327171.csv', '306183.csv', '312900.csv',
 '311548.csv', '345496.csv', '319921.csv', '335372.csv', '317461.csv', '319699.csv', '321448.csv',
 '311951.csv', '345454.csv', '314021.csv', '322578.csv', '304352.csv', '335992.csv', '330829.csv',
 '335414.csv', '345124.csv', '345520.csv', '311944.csv', '328690.csv', '332569.csv', '330381.csv',
 '345736.csv', '316356.csv', '327809.csv', '331058.csv', '335216.csv', '307298.csv', '324327.csv',
 '374074.csv', '305896.csv', '330811.csv', '331272.csv', '345322.csv', '328864.csv', '345256.csv',
 '305672.csv', '335570.csv', '305706.csv', '335612.csv', '305060.csv', '326108.csv', '345108.csv',
 '335174.csv', '305920.csv', '327411.csv', '312736.csv', '302901.csv', '320747.csv', '345652.csv',
 '345280.csv', '329789.csv', '321471.csv', '305854.csv', '331298.csv', '326280.csv', '315689.csv',
 '345041.csv', '303982.csv', '303016.csv', '326082.csv', '335968.csv', '313239.csv', '327163.csv',
 '315853.csv', '335836.csv', '310268.csv', '323675.csv', '345447.csv', '310532.csv', '330027.csv',
 '374314.csv', '310862.csv', '345686.csv', '335349.csv', '302240.csv', '326447.csv', '325167.csv',
 '328443.csv', '313833.csv', '345645.csv', '308585.csv', '333088.csv', '305102.csv', '345678.csv',
 '376517.csv', '335638.csv', '335604.csv', '376715.csv', '335943.csv', '311639.csv', '318345.csv',
 '318444.csv', '319772.csv', '336008.csv', '319202.csv', '345025.csv', '321992.csv',
 '304535.csv', '335844.csv', '314054.csv', '309005.csv', '345421.csv', '328815.csv', '305987.csv', '311845.csv', '308247.csv', '311266.csv', '330928.csv',
 '328419.csv', '322479.csv', '331439.csv', '324020.csv', '309617.csv', '335893.csv', '323410.csv', '335851.csv', '328021.csv',
 '321950.csv', '306928.csv', '335448.csv', '323766.csv', '314294.csv', '335299.csv', '345595.csv', '376954.csv', '300236.csv', '326595.csv', '316091.csv', '335927.csv', '303503.csv', '305953.csv', '301937.csv', '308668.csv', '325894.csv', '306043.csv', '322255.csv', '335729.csv',
 '305946.csv', '335661.csv', '335885.csv', '325738.csv', '325274.csv', '319764.csv', '326231.csv', '335067.csv', '321984.csv', '318485.csv', '306456.csv', '329847.csv',
 '309211.csv', '301846.csv', '323832.csv', '319947.csv', '330480.csv', '345355.csv', '303248.csv', '305995.csv', '306522.csv', '345553.csv', '335117.csv', '345157.csv', '318873.csv', '335315.csv', '326355.csv', '323204.csv', '313304.csv', '345397.csv', '309823.csv', '345181.csv',
'306084.csv', '323774.csv', '313338.csv', '301648.csv', '313700.csv', '330456.csv', '345744.csv', '326181.csv', '329649.csv', '321836.csv', '314641.csv', '313064.csv', '345546.csv', '302380.csv', '305821.csv', '345587.csv', '335869.csv', '323428.csv', '326140.csv', '327114.csv', '326975.csv', '303263.csv', '328435.csv', '306092.csv', '300384.csv', '335471.csv', '304691.csv', '345140.csv', '316083.csv', '319830.csv', '313460.csv', '345579.csv', '303909.csv', '324434.csv', '335539.csv', '300780.csv', '300970.csv', '326223.csv', '345223.csv', '314534.csv', '304733.csv', '323550.csv', '330910.csv', '335497.csv', '300996.csv', '335695.csv', '305540.csv', '338616.csv', '325928.csv', '323154.csv', '328773.csv', '345603.csv', '325134.csv', '335455.csv', '331223.csv', '308924.csv', '315960.csv', '302992.csv', '345165.csv', '379313.csv', '319418.csv', '322115.csv', '312611.csv', '345561.csv', '319236.csv', '321554.csv', '335737.csv', '335521.csv', '345207.csv', '315168.csv', '316661.csv', '304071.csv', '375113.csv', '317594.csv', '301309.csv', '346056.csv', '309609.csv', '345371.csv', '319426.csv', '307793.csv', '345629.csv', '300822.csv', '305755.csv', '329268.csv', '306274.csv', '345173.csv', '304675.csv', '330274.csv', '335331.csv', '307355.csv', '326371.csv', '302588.csv', '307397.csv', '327536.csv', '316265.csv', '345199.csv', '330738.csv', '374967.csv', '331397.csv', '326199.csv', '323618.csv', '335091.csv', '319962.csv', '317230.csv', '305813.csv', '328211.csv', '307157.csv', '345348.csv', '335877.csv', '313668.csv', '315580.csv', '310995.csv', '307769.csv', '304923.csv', '375923.csv', '321778.csv', '326413.csv', '376608.csv', '309427.csv', '335679.csv', '317396.csv', '309625.csv', '345413.csv', '304063.csv', '330852.csv', '345017.csv', '345215.csv', '301127.csv', '303917.csv', '312165.csv', '309021.csv', '309948.csv', '335109.csv', '319434.csv', '345363.csv', '314104.csv', '345405.csv', '345439.csv', '306266.csv', '313643.csv', '321752.csv', '309419.csv', '326439.csv', '324038.csv', '321430.csv', '322313.csv', '335323.csv', '316505.csv', '335653.csv', '300616.csv', '329078.csv', '326389.csv', '301440.csv', '345389.csv', '328401.csv', '310607.csv', '377887.csv', '335646.csv', '300400.csv', '315390.csv', '300665.csv', '310029.csv', '322925.csv', '335901.csv', '303727.csv', '309906.csv', '328260.csv', '309641.csv', '327437.csv', '319541.csv', '345488.csv', '322610.csv', '309872.csv', '335422.csv', '325142.csv', '329037.csv', '326462.csv', '319889.csv', '305292.csv', '345728.csv', '326264.csv', '335620.csv', '304600.csv', '313817.csv', '345462.csv', '345660.csv', '345264.csv', '335026.csv', '345272.csv', '345058.csv', '313544.csv', '335554.csv', '307082.csv', '345066.csv', '329169.csv', '302307.csv', '306613.csv', '326306.csv', '314161.csv', '323295.csv', '312504.csv', '304170.csv', '323097.csv', '306001.csv', '335018.csv', '345298.csv', '304428.csv', '327031.csv', '328666.csv', '305904.csv', '323055.csv', '328302.csv', '330001.csv', '335810.csv', '305938.csv', '323121.csv', '307520.csv', '326298.csv', '335596.csv', '301838.csv', '331082.csv', '329144.csv', '313569.csv', '300111.csv', '326934.csv', '335182.csv', '327551.csv', '311381.csv', '307124.csv', '306399.csv', '335828.csv', '331124.csv', '328104.csv', '326868.csv', '310672.csv', '312918.csv', '335430.csv', '323253.csv', '335141.csv', '305524.csv', '321646.csv', '376657.csv', '317487.csv', '345504.csv', '318824.csv', '309054.csv', '316000.csv', '345470.csv', '326074.csv', '304410.csv', '306167.csv', '306415.csv', '316174.csv', '325019.csv', '316612.csv', '326504.csv', '312660.csv', '345538.csv', '335208.csv', '317446.csv', '345512.csv', '326843.csv', '335778.csv', '322990.csv', '345710.csv', '345116.csv', '323709.csv',
  '331330.csv', '319053.csv', '374678.csv', '319442.csv', '345314.csv', '313387.csv', '305268.csv', '305862.csv', '304956.csv', '326249.csv', '329813.csv', '301812.csv', '335976.csv', '345249.csv', '327619.csv', '335547.csv']

In [12]:
# Copy to a new folder 487 sampling points

os.mkdir(folder_path_487)

for i in points_data_487:
  old_path=os.path.join(cleaned_output_folder, i)
  new_path=os.path.join(folder_path_487, i)
  shutil.copyfile(old_path, new_path)

## **Feature Extraction Technique with LightGBM Model**

In [13]:
# Instantiate Model
model_name="LightGBM"
model=LGBMRegressor(verbose=-1)

# Create empty lists
Pred=[]
actuals=[]

# Loop through each csv file
for i in os.listdir(folder_path_487):

  # Read the csv file
  path=os.path.join(folder_path_487, i)
  df=pd.read_csv(path)

  # Remove last row and fill na's
  df=df.iloc[:-1,:]
  df.ffill(inplace=True)

  # Convert date to datetime feature
  df["Date"]=pd.to_datetime(df["Date"], format="%d.%m.%Y %H:%M:%S   ")
  df.sort_values(by="Date", inplace=True)

  # Extract datetime features
  df["Day"]=df["Date"].dt.day
  df["Dayofweek"]=df["Date"].dt.dayofweek
  df["Month"]=df["Date"].dt.month
  df["Year"]=df["Date"].dt.year
  df["Quarter"]=df["Date"].dt.quarter
  df["Week"]=df["Date"].dt.isocalendar().week
  df=df.drop("Date", axis=1)

  # Split data to train and test
  train=df.iloc[:-26,:]
  test=df.iloc[-26:, :]

  # Split data to dependent and independent variables
  X_train=train.drop("Levels", axis=1)
  y_train=train["Levels"]
  X_test=test.drop("Levels", axis=1)
  y_test=test["Levels"]

  # Train model
  model.fit(X_train, y_train)
  pred=model.predict(X_test)

  # Test the model
  for yss in y_test.values:
    actuals.append(yss)
  for yss in pred:
    Pred.append(yss)

# Create dataframe for y_tests and corresponding Predictions
test=pd.DataFrame({"Predictions":Pred, "Actuals":actuals}).dropna()

# Calculate SMAPE Score
A = test["Actuals"]
F = test["Predictions"]
print(model_name+" SMAPE Score: ", smape(A, F))

LightGBM SMAPE Score:  0.12082219218344961


## **Forecast for 2022 - 2024 Feb**

In [14]:
# Create a date range starting from 01.01.2022 with monthly frequency
date_range = pd.date_range(start='2022-01-01', end='2024-02-01', freq='MS')

# Create a DataFrame
df_ = pd.DataFrame(date_range, columns=['Date'])

# Extract Features
df_["Day"]=df_["Date"].dt.day
df_["Dayofweek"]=df_["Date"].dt.dayofweek
df_["Month"]=df_["Date"].dt.month
df_["Year"]=df_["Date"].dt.year
df_["Quarter"]=df_["Date"].dt.quarter
df_["Week"]=df_["Date"].dt.isocalendar().week

df_.drop("Date", axis=1, inplace=True)

In [15]:
Pred=[]

# Loop through each CSV file
for i in os.listdir(folder_path_487):

    # Read CSV, drop 2022 and fill na
    path=os.path.join(folder_path_487, i)
    df=pd.read_csv(path)
    df=df.iloc[:-1,:]
    df.ffill(inplace=True)

    # Change Date column to datetime features
    df["Date"]=pd.to_datetime(df["Date"], format="%d.%m.%Y %H:%M:%S   ")
    df.sort_values(by="Date", inplace=True)

    # Extract Date features
    df["Day"]=df["Date"].dt.day
    df["Dayofweek"]=df["Date"].dt.dayofweek
    df["Month"]=df["Date"].dt.month
    df["Year"]=df["Date"].dt.year
    df["Quarter"]=df["Date"].dt.quarter
    df["Week"]=df["Date"].dt.isocalendar().week
    df=df.drop("Date", axis=1)

    # Split to dependent and independent variables
    X=df.drop("Levels", axis=1)
    y=df["Levels"]

    # Train model
    model=LGBMRegressor(verbose=-1)
    model.fit(X, y)

    # Make forecast
    pred=model.predict(df_)

    # Append forecast and site id to Pred list
    Pred.append([i.split(".")[0], pred])

# Generate dataframe for forecasted data
preds=pd.DataFrame(Pred)
sites=preds[0].values
preds=pd.DataFrame(preds[1].to_list())
preds['Sites']=sites
preds.set_index('Sites', inplace=True)
preds=preds.T
preds["Date"] = date_range
preds.set_index("Date", inplace=True)
preds=preds.round(2)
preds
preds.to_csv("groundwater_forecasts.csv")

## **Train and Make Prediction For Specific Geographic Location**

In [16]:
# Define the file path for the training data
file_path = "/content/Grundwasserstand-Monatsmittel/300111.csv"  # Change to the specific data file path
# Define the start and end dates for the forecast
start_date = "2022-01-01"  # Change to desired start date
end_date = "2024-02-01"    # Change to desired end date


In [17]:
# Import libraries
import pandas as pd
import numpy as np
import os
import shutil
from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

In [18]:
def process_file(input_file, output_file, encoding='ISO-8859-1'):
    # Read the input file with a specific encoding
    try:
        with open(input_file, 'r', encoding=encoding) as f:
            lines = f.readlines()
    except UnicodeDecodeError:
        print(f"Error decoding {input_file} with encoding {encoding}.")
        return

    # Find the line starting with 'Werte:'
    start_index = None
    for i, line in enumerate(lines):
        if line.startswith('Werte:'):
            start_index = i
            break

    if start_index is None:
        print(f"Warning: 'Werte:' not found in {input_file}.")
        return

    relevant_lines = lines[start_index + 1:]

    with open(output_file, 'w', encoding=encoding) as f:
        f.writelines(relevant_lines)

    print(f"Processed {input_file} and saved to {output_file}.")


def clean_data(file_path):

  file_name=file_path

  process_file(file_name, 'cleaned_input.csv', encoding='ISO-8859-1')

  df=pd.read_csv("cleaned_input.csv", encoding='latin-1', header=None)
  df["data"]=df[0].str.split(';')
  df_=pd.DataFrame(df["data"].to_list(), columns=['Date', 'Data1', 'Data2'])
  df[1]=df[1].str.strip(' ;')
  df_[1]=df[1]
  df_["Levels"]=df_["Data1"]+"."+df_[1]
  df_=df_[["Date","Levels"]]
  df_["Levels"]=df_["Levels"].astype("float64")
  return df_

In [19]:
def make_forecast_dates(start_date = "2022-01-01", end_date="2024-02-01"):

  # Create a date range starting from start date with monthly frequency
  date_range = pd.date_range(start=start_date, end=end_date, freq='MS')

  # Create a DataFrame
  df_ = pd.DataFrame(date_range, columns=['Date'])

  # Extract Date Features from DateFrame
  df_["Day"]=df_["Date"].dt.day
  df_["Dayofweek"]=df_["Date"].dt.dayofweek
  df_["Month"]=df_["Date"].dt.month
  df_["Year"]=df_["Date"].dt.year
  df_["Quarter"]=df_["Date"].dt.quarter
  df_["Week"]=df_["Date"].dt.isocalendar().week

  return df_

In [20]:
def train_model(data_frame):

  # Read in the dataset for specific loactaion
  df=data_frame

  # Remove 2022 row
  df=df.iloc[:-1,:]

  # Fill missing values
  df.ffill(inplace=True)

  # Convert date column to datetime type
  df["Date"]=pd.to_datetime(df["Date"], format="%d.%m.%Y %H:%M:%S   ")
  df.sort_values(by="Date", inplace=True)

  # Extract Features
  df["Day"]=df["Date"].dt.day
  df["Dayofweek"]=df["Date"].dt.dayofweek
  df["Month"]=df["Date"].dt.month
  df["Year"]=df["Date"].dt.year
  df["Quarter"]=df["Date"].dt.quarter
  df["Week"]=df["Date"].dt.isocalendar().week

  # Split data to dependent and independent variables
  df=df.drop("Date", axis=1)
  X=df.drop("Levels", axis=1)
  y=df["Levels"]

  # Train and return the model
  model=LGBMRegressor(verbose=-1)
  model.fit(X, y)
  return model

In [21]:
def make_forecasts(model, forecast_dates):

  forecasts=pd.DataFrame()

  # Make Predictions for forecast_dates
  forecasts["Date"]= forecast_dates["Date"]
  forecast_dates.drop("Date", axis=1, inplace=True)
  pred=model.predict(forecast_dates)
  forecasts["Levels"]=pred
  forecasts=forecasts.round(2)

  return forecasts

In [22]:
# Generate DataFrame for forecast dates
forecast_dates = make_forecast_dates(start_date=start_date, end_date=end_date)

# Clean dataset
df_ = clean_data(file_path)

# Train the model using the specified data file
model = train_model(df_)

# Make forecasts using the trained model and forecast dates
forecasts = make_forecasts(model, forecast_dates)

# Optionally, print or save the forecasts
print(forecasts)
forecasts.to_csv("forecasts.csv", index=False)
print("\nFORECASTS SAVED SUCCESSFULLY")

Processed /content/Grundwasserstand-Monatsmittel/300111.csv and saved to cleaned_input.csv.
         Date  Levels
0  2022-01-01  154.02
1  2022-02-01  153.76
2  2022-03-01  153.77
3  2022-04-01  153.75
4  2022-05-01  153.75
5  2022-06-01  153.86
6  2022-07-01  154.04
7  2022-08-01  154.06
8  2022-09-01  154.04
9  2022-10-01  154.02
10 2022-11-01  154.03
11 2022-12-01  154.03
12 2023-01-01  154.05
13 2023-02-01  153.76
14 2023-03-01  153.77
15 2023-04-01  153.75
16 2023-05-01  153.81
17 2023-06-01  153.86
18 2023-07-01  154.04
19 2023-08-01  154.05
20 2023-09-01  154.03
21 2023-10-01  154.03
22 2023-11-01  154.03
23 2023-12-01  154.01
24 2024-01-01  153.68
25 2024-02-01  153.76

FORECASTS SAVED SUCCESSFULLY
